In [1]:
#Imports
import keras
import tensorflow as tf
from keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

#Presets
num_images = 4000

#Import data—data is local
data = pd.read_csv('train.csv')
images = data.iloc[0:num_images,1:]
train_images = images / 255.0
train_images = train_images.values.reshape(num_images, 28, 28, 1)
train_labels = data.iloc[0:num_images,:1]

train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, train_size = .25, random_state=42)

#A class holding the 'genes' of a neural network
class Metadata:
    def __init__(self, dense_layers, dense_layer_nodes, kernels_0, kernels_1, model_type, ancestry):
        self.dense_layers = dense_layers
        self.dense_layer_nodes = dense_layer_nodes
        self.kernels_0 = kernels_0
        self.kernels_1 = kernels_1
        self.model_type = model_type #'bred' or 'mutated'—how the model was formed
        self.ancestry = ancestry #0=seeded, 1=random, values in between=mix of seeded and random (ie. if a network is
                                 #is descened from three randoms and 1 seeded, ancestry=.75)
        
    def print_metadata(self):
        print(self.dense_layers, self.dense_layer_nodes, self.kernels_0, self.kernels_1, self.model_type, self.ancestry)

#A class comprising a neural network model, a Metadata object with the model's genes,
#the network's accuracy, and the place it finished in
class ModelWithMetadata:
    def __init__(self, metadata):
        self.model = self.get_model_from_seed(metadata)
        self.metadata = metadata
        self.accuracy = self.set_accuracy(self.model)
        self.place = -1 #A place set to -1 indicates that it is not yet set
        print("Trained model with accuracy ", self.accuracy)
        self.metadata.print_metadata()
    
    #Sets the place a model finished in for a given generation. 1 = best model, 5 = fifth best
    def set_place(self, place):
        self.place = place
    
    #Sets the model's accuracy by running it
    def set_accuracy(self, model):
        try:
            model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(train_images, train_labels, batch_size=100, epochs=10, verbose=0, validation_data=(validation_images, validation_labels))
            return model.evaluate(validation_images, validation_labels)[1]
        except:
            print("Model failed")
            return 0.0
    
    #Returns a model, given a Metadata object
    def get_model_from_seed(self, metadata):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(metadata.kernels_0, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Conv2D(metadata.kernels_1, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Flatten())
        for i in range(metadata.dense_layers):
            model.add(keras.layers.Dense(metadata.dense_layer_nodes[i], activation="relu", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        model.add(keras.layers.Dense(10, activation="softmax", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        return model
    
    #Returns a model similar to the one the parameter metadata came from.
    #Analogous to asexual reproduction
    def mutate(metadata):
        dense_layers = 3 #Having different numbers of layers is more to deal with, and not a terrible invariant
        dense_layer_nodes = list()
        for i in range(dense_layers):
            if (i < len(metadata.dense_layer_nodes)):
                dense_layer_nodes.append(max(10, metadata.dense_layer_nodes[i] + random.randint(-10,10)))
            else:
                dense_layer_nodes.append(random.randint(10,1001))
        kernels_0 = max(10, metadata.kernels_0 + random.randint(-10, 10))
        kernels_1 = max(10, metadata.kernels_1 + random.randint(-10, 10))
        return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1, "mutated", metadata.ancestry))

    #Returns a model similar to the ones the Metadata parameters m1 and m2 came from.
    #Analogous to sexual reproduction
    def breed(m1, m2):
        dense_layers = 3
        dense_layer_nodes = m1.dense_layer_nodes if random.getrandbits(1) else m2.dense_layer_nodes
        kernels_0 = m1.kernels_0 if random.getrandbits(1) else m2.kernels_0
        kernels_1 = m1.kernels_1 if random.getrandbits(1) else m2.kernels_1
        ancestry = (m1.ancestry + m2.ancestry) / 2
        return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1, "bred", ancestry))

Using TensorFlow backend.
/Users/tristanengst/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
#Models evolution of the population of neural networks comprising the elements of models over num_generations
def evolve(num_generations, models):
    best_accuracies = []
    average_accuracies = []
    worst_accuracies = []
    types_timeline = []
    ancestry_timeline = []
    better_than_fifth = []
    better_than_third = []
    better_than_first = []
    print("Commenced evolution after generation zero!")
    for i in range(num_generations - 1):
        #Get and record the accuracies of the generation
        models.sort(key=lambda model: model.accuracy) #The best model has the highest index
        generation_accuracy = 0
        for model in models:
            generation_accuracy += model.accuracy / len(models)
        best_accuracies.append(models[len(models) - 1].accuracy)
        average_accuracies.append(generation_accuracy)
        worst_accuracies.append(models[0].accuracy)
        
        #Find the number of NEW or PERSISTED models that are better than those 5th best, 3rd best,
        #and best of the previous generation. This could be interesting.
        bt5 = 0
        bt3 = 0
        bt1 = 0
        for j in range(len(models)):
            if models[j].place == 5:
                for k in range(j + 1, len(models)):
                    if models[k].place == -1:
                        bt5 += 1
            if models[j].place == 3:
                for k in range(j + 1, len(models)):
                    if models[k].place == -1:
                        bt3 += 1
            if models[j].place == 31:
                for k in range(j + 1, len(models)):
                    if models[k].place == -1:
                        bt1 += 1
        better_than_fifth.append(bt5)
        better_than_third.append(bt3)
        better_than_first.append(bt1)
        
        #Set the places of the top five models, and record their types ('mutated' or 'bred')
        #and ancestry ('seeded', 'mixed', or 'random')
        types = []
        ancestry = []
        for j in range(5):
            models[len(models) - 1 - j].place = j + 1
            types.append(models[len(models) - 1 - j].metadata.model_type)
            ancestry.append(models[len(models) - 1 - j].metadata.ancestry)
        types_timeline.append(types)
        ancestry_timeline.append(ancestry)
        
        print("Worst accuracy of generation:", models[0].accuracy, "\n",
                  "Average accuracy:", generation_accuracy, "\n",
                  "Best accuracy of generation:", models[len(models) - 1].accuracy, "\n",
                  "Average accuracy:", generation_accuracy, "\n",
                  "Better than fifth:", bt5, "\n",
                  "Better than third:", bt3, "\n",
                  "Better than first:", bt1, "\n",
                  "Types (1st place first):", types, "\n",
                  "Ancestry (1st place first):", ancestry, "\n")
        
        if (i != num_generations - 2):
            #Delete all but the best five models and two randomly chosen ones from outside the best five
            saved_1 = models[random.randint(0, len(models) - 1 - 5)]
            saved_2 = models[random.randint(0, len(models) - 1 - 5)]
            while (len(models) > 5):
                models.pop(0)
            models.append(saved_1)
            models.append(saved_2)
            #Models has seven elements with the first five being ordered in increasing accuracy
        
            #Add asexual models. The loops are to ease further fiddling
            for j in range(1):
                model = ModelWithMetadata(ModelWithMetadata.mutate(models[2].metadata))
                models.append(model)
            for j in range(2):
                model = ModelWithMetadata(ModelWithMetadata.mutate(models[3].metadata))
                models.append(model)
            for j in range(4):
                model = ModelWithMetadata(ModelWithMetadata.mutate(models[4].metadata))
                models.append(model)
            #Models has 14 elements
        
            #Add sexual models from the UPDATED best 3
            models.sort(reverse=True, key=lambda model: model.accuracy) #Best model is at index 0
            models.append(ModelWithMetadata(ModelWithMetadata.breed(models[0].metadata, models[1].metadata)))
            models.append(ModelWithMetadata(ModelWithMetadata.breed(models[0].metadata, models[2].metadata)))
            models.append(ModelWithMetadata(ModelWithMetadata.breed(models[1].metadata, models[2].metadata)))
        
            #Add sexual models from the best 3 and randomly selected ones
            for i in range(4):
                rand_1 = random.randint(0, 2)
                rand_2 = random.randint(3, len(models) - 1)
                models.append(ModelWithMetadata(ModelWithMetadata.breed(models[rand_1].metadata, models[rand_2].metadata)))
            
            #Models has 21 elements
    return best_accuracies, average_accuracies, worst_accuracies, types_timeline, ancestry_timeline, better_than_fifth, better_than_third, better_than_first



In [3]:
print("This doesn't seem to do anything for a while. Hang with it.")
models = []

#Here I'm adding "seeds" for various classes of models that tend to work well. This is to fix a problem where none, or not the best of these, were initially developed,
#leading to the model converging towards a local and not (potentially) global maximum accuracy
for i in range(1):
    models.append(ModelWithMetadata(Metadata(3, [random.randint(750,1000), random.randint(750,1000), random.randint(750,1000)], random.randint(1,50), random.randint(1,50), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(500,1000), random.randint(500,1000), random.randint(500,1000)], random.randint(1,50), random.randint(1,50), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(500,1000), random.randint(500,1000), random.randint(500,1000)], random.randint(50,100), random.randint(100,200), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(250,1000), random.randint(250,1000), random.randint(250,1000)], random.randint(1,50), random.randint(1,50), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(250,1000), random.randint(250,1000), random.randint(250,1000)], random.randint(50,200), random.randint(50,200), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200), "", 0)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200), "", 0)))
evolve(2, models)
    
#Here we add more models than we typically would so that generation zero is very diverse
for i in range(10):
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200), "", 1)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(750,1000), random.randint(750,1000), random.randint(750,1000)], random.randint(50,200), random.randint(50,200), "", 1)))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(500,1000), random.randint(500,1000), random.randint(500,1000)], random.randint(50,200), random.randint(50,200), "", 1)))

This doesn't seem to do anything for a while. Hang with it.
3000/3000 [==============================] - 1s 451us/step
Trained model with accuracy  0.9276666666666666
3 [968, 863, 833] 35 42
3000/3000 [==============================] - 1s 466us/step
Trained model with accuracy  0.8989999998410543
3 [811, 643, 849] 50 9
3000/3000 [==============================] - 3s 1ms/step
Trained model with accuracy  0.921666666507721
3 [811, 611, 589] 89 155
3000/3000 [==============================] - 1s 281us/step
Trained model with accuracy  0.9083333333333333
3 [891, 763, 324] 22 11
3000/3000 [==============================] - 4s 1ms/step
Trained model with accuracy  0.93
3 [692, 439, 498] 106 180
3000/3000 [==============================] - 5s 2ms/step
Trained model with accuracy  0.9173333333333333
3 [766, 484, 311] 150 92
3000/3000 [==============================] - 2s 583us/step
Trained model with accuracy  0.9233333333333333
3 [811, 858, 307] 48 186
3000/3000 [=============================

NameError: name 'bt4' is not defined

In [10]:
print("running")
best, average, worst, types, ancestry, bt5, bt3, bt1 = evolve(3, models)

running
Commenced evolution after generation zero!
Worst accuracy of generation: 0.9053333334922791 
 Average accuracy: 0.9242857142932831 
 Best accuracy of generation: 0.939 
 Average accuracy: 0.9242857142932831 
 Better than fifth: 2 
 Better than third: 1 
 Better than first: 0 
 Types (1st place first): ['mutated', 'mutated', 'bred', 'mutated', 'mutated'] 
 Ancestry (1st place first): [0, 0, 0.0, 0, 0] 

3000/3000 [==============================] - 2s 629us/step
Trained model with accuracy  0.9263333333333333
3 [967, 867, 845] 37 54
3000/3000 [==============================] - 2s 612us/step
Trained model with accuracy  0.929
3 [950, 870, 837] 39 50
3000/3000 [==============================] - 2s 618us/step
Trained model with accuracy  0.9226666666666666
3 [961, 865, 848] 31 41
3000/3000 [==============================] - 2s 565us/step
Trained model with accuracy  0.9303333333333333
3 [974, 863, 825] 35 43
3000/3000 [==============================] - 2s 708us/step
Trained model wi

In [ ]:
def is_it_evolution(generations, models):
    best_accuracies = []
    average_accuracies = []
    worst_accuracies = []
    for i in range(generations - 1):
        for j in range(7):
            models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200))))
            models.append(ModelWithMetadata(Metadata(3, [random.randint(1,500), random.randint(1,500), random.randint(1,500)], random.randint(1,100), random.randint(1,100))))
            models.append(ModelWithMetadata(Metadata(3, [random.randint(1,250), random.randint(1,250), random.randint(1,250)], random.randint(1,50), random.randint(1,50))))
        models.sort(key=lambda model: model.accuracy)
        generation_accuracy = 0
        for model in models:
            generation_accuracy += model.accuracy / len(models)
        print("Worst accuracy of generation:", models[0].accuracy, "Best accuracy of generation:", models[len(models) - 1].accuracy, "Average accuracy:", generation_accuracy)
        models[len(models) - 1].metadata.print_metadata()
        best_accuracies.append(models[len(models) - 1].accuracy)
        average_accuracies.append(generation_accuracy)
        worst_accuracies.append(models[0].accuracy)
        saved_1 = models[random.randint(0, len(models) - 1 - 3)]
        saved_2 = models[random.randint(0, len(models) - 1 - 3)]
        #Delete all but the best three models and the append the two saved ones
        while (len(models) > 3):
            models.pop(0)
        models.append(saved_1)
        models.append(saved_2)
    return best_accuracies, average_accuracies, worst_accuracies

models.clear()
for j in range(7):
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,1000), random.randint(1,1000), random.randint(1,1000)], random.randint(1,200), random.randint(1,200))))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,500), random.randint(1,500), random.randint(1,500)], random.randint(1,100), random.randint(1,100))))
    models.append(ModelWithMetadata(Metadata(3, [random.randint(1,250), random.randint(1,250), random.randint(1,250)], random.randint(1,50), random.randint(1,50))))



In [ ]:
best_nonevolutionary, average_nonevolutionary, worst_nonevolutionary = is_it_evolution(40, models)